In [1]:
import numpy as np
import pandas as pd
from superlink.superlink import SuperLink
import superlink.geometry
from data_2 import *
import scipy.optimize
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette('husl')
%matplotlib inline

In [2]:
superjunctions = pd.DataFrame.from_dict(superjunctions, orient='index')
superlinks = pd.DataFrame.from_dict(superlinks, orient='index')
links = pd.concat({k: pd.DataFrame.from_dict(v, orient='index')
                   for k,v in links.items()}, axis=0)
junctions = pd.concat({k: pd.DataFrame.from_dict(v, orient='index')
                       for k,v in junctions.items()}, axis=0)

links = links.reset_index().set_index('level_1')
junctions = junctions.reset_index().set_index('level_1')

superjunctions['h_0'] += 1e-5
junctions['h_0'] += 1e-5
links['Q_0'] += 0

In [3]:
del links['w']

In [4]:
geom = {
    0 : 'CIRCULAR',
    1 : 'RECT_CLOSED',
    2 : 'IRREGULAR',
    3 : 'TRIANGULAR',
    4 : 'TRAPEZOIDAL',
    5 : 'IRREGULAR'
}

g1 = {
    0 : 0.8,
    1 : 0.5,
    2 : 0,
    3 : 0.5,
    4 : 0.5,
    5 : 0
}

g2 = {
    0 : 0,
    1 : 0.5,
    2 : 0,
    3 : 0.5,
    4 : 0.5,
    5 : 0
}

g3 = {
    0 : 0,
    1 : 0,
    2 : 0,
    3 : 0,
    4 : 1,
    5 : 0
}

ts = {
    0 : None,
    1 : None,
    2 : 't1',
    3 : None,
    4 : None,
    5 : 't2'
}

In [5]:
transects = {
    't1' : {
        'x' : 0.2 * np.array([0, 1, 1.5, 2.5, 3, 4], dtype=float),
        'y' : 0.5 * np.array([1, 0.8, 0, 0, 0.8, 1], dtype=float),
        'horiz_points' : 100,
        'vert_points' : 100
    },
    't2' : {
        'x' : 0.3 * np.array([0, 1, 1, 2, 2, 3], dtype=float),
        'y' : 0.5 * np.array([1, 0.8, 0, 0, 0.8, 1], dtype=float),
        'horiz_points' : 100,
        'vert_points' : 100
    },
}

In [6]:
transect_factory = {}

for transect_name, transect in transects.items():
    transect_factory[transect_name] = superlink.geometry.Irregular(**transect)

In [7]:
links['geom'] = links['level_0'].map(geom)
links['g1'] = links['level_0'].map(g1)
links['g2'] = links['level_0'].map(g2)
links['g3'] = links['level_0'].map(g3)
links['ts'] = links['level_0'].map(ts)

In [8]:
is_irregular = links['geom'].str.lower() == 'irregular'
irregular_links = links[is_irregular]
transect_indices = pd.Series(irregular_links.index, index=irregular_links['ts'].values)

In [9]:
unique_geom = set(links['geom'].str.lower().unique())
has_irregular = ('irregular' in unique_geom)
unique_geom.discard('irregular')

In [10]:
regular_links = links[~is_irregular]
geom_indices = pd.Series(regular_links.index, index=regular_links['geom'].str.lower().values)

In [11]:
_A_ik = np.zeros(len(links))
_Pe_ik = np.zeros(len(links))
_R_ik = np.zeros(len(links))
_B_ik = np.zeros(len(links))

In [12]:
for geom in unique_geom:
    Geom = geom.title()
    ik = geom_indices.loc[geom].values
    ########## TODO: Temporary
    h_Ik = np.full(ik.size, 1e-5)
    h_Ip1k = np.full(ik.size, 1e-5)
    ##########
    generator = getattr(superlink.geometry, Geom)
    g1 = links.loc[ik, 'g1'].values
    g2 = links.loc[ik, 'g2'].values
    g3 = links.loc[ik, 'g3'].values
    _A_ik[ik] = generator.A_ik(h_Ik, h_Ip1k, g1=g1, g2=g2, g3=g3)
    _Pe_ik[ik] = generator.Pe_ik(h_Ik, h_Ip1k, g1=g1, g2=g2, g3=g3)
    _R_ik[ik] = generator.R_ik(_A_ik[ik], _Pe_ik[ik])
    _B_ik[ik] = generator.B_ik(h_Ik, h_Ip1k, g1=g1, g2=g2, g3=g3)

In [13]:
for transect_name, generator in transect_factory.items():
    ik = transect_indices.loc[transect_name].values
    ########## TODO: Temporary
    h_Ik = np.full(ik.size, 1e-5)
    h_Ip1k = np.full(ik.size, 1e-5)
    ##########
    _A_ik[ik] = generator.A_ik(h_Ik, h_Ip1k)
    _Pe_ik[ik] = generator.Pe_ik(h_Ik, h_Ip1k)
    _R_ik[ik] = generator.R_ik(h_Ik, h_Ip1k)
    _B_ik[ik] = generator.B_ik(h_Ik, h_Ip1k)

In [14]:
_A_ik

array([3.77122202e-08, 3.77122202e-08, 3.77122202e-08, 5.00000000e-06,
       5.00000000e-06, 5.00000000e-06, 1.97171717e-06, 1.97171717e-06,
       1.97171717e-06, 5.00000000e-11, 5.00000000e-11, 5.00000000e-11,
       5.00010000e-06, 5.00010000e-06, 5.00010000e-06, 3.00000000e-06,
       3.00000000e-06, 3.00000000e-06])

In [15]:
_Pe_ik

array([5.65686603e-03, 5.65686603e-03, 5.65686603e-03, 5.00020000e-01,
       5.00020000e-01, 5.00020000e-01, 4.20510954e-04, 4.20510954e-04,
       4.20510954e-04, 2.23606798e-05, 2.23606798e-05, 2.23606798e-05,
       5.00028284e-01, 5.00028284e-01, 5.00028284e-01, 6.05363083e-04,
       6.05363083e-04, 6.05363083e-04])

In [16]:
_R_ik

array([6.66662778e-06, 6.66662778e-06, 6.66662778e-06, 9.99960002e-06,
       9.99960002e-06, 9.99960002e-06, 9.28394364e-06, 9.28394364e-06,
       9.28394364e-06, 2.23606798e-06, 2.23606798e-06, 2.23606798e-06,
       9.99963434e-06, 9.99963434e-06, 9.99963434e-06, 9.81229309e-06,
       9.81229309e-06, 9.81229309e-06])

In [17]:
_B_ik

array([5.65681889e-03, 5.65681889e-03, 5.65681889e-03, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 1.93971394e-01, 1.93971394e-01,
       1.93971394e-01, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
       5.00020000e-01, 5.00020000e-01, 5.00020000e-01, 3.00000000e-01,
       3.00000000e-01, 3.00000000e-01])